# Imports

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tlbo import TLBO


from dataclasses import dataclass, field
from typing import Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import multiprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, make_scorer, log_loss
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.optimizers import RMSprop
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import warnings
warnings.filterwarnings('ignore')

2024-11-10 21:54:58.154485: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 21:54:58.155529: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 21:54:58.158596: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 21:54:58.168844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 21:54:58.184888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# SVM

## Homem 1

In [4]:
def run_model(model, X_train, y_train, X_test, y_test, metric):
    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)


    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy

def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)
    # model_svm = SVC(kernel="linear", C=1, probability=True)

    (scores_svm, mean_loss_svm, test_accuracy) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test,
                                            metric=metric)
    
    features = list(chromossome).count(1)

    if metric == 'train':
        return mean_loss_svm + 0.001*features
    
    elif metric == 'validation':

        print(f'Accuracy in Training: {mean_loss_svm}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_svm


In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.95,
            iterations_size=20)
 
best_student, mean_fitness, mean_features  = tlbo.execute()

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [01:17<00:00,  3.90s/it]


Fitness: 1.055 | features: 38
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:59<00:00,  2.98s/it]


Fitness: 0.814 | features: 38
Iteration: 3


...

Fitness: 0.805 | features: 43
Iteration: 19


Calculating Population Fitness: 100%|██████████| 20/20 [00:40<00:00,  2.02s/it]


Fitness: 0.805 | features: 43
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Fitness: 0.805 | features: 43


In [6]:
evaluate_function(best_student.position, metric = 'validation')

Accuracy in Training: 0.6987
Accuracy in Validation: 0.6979


0.6987

## Homem 2

In [7]:
def run_model(model, X_train, y_train, X_test, y_test, metric):
    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)


    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy

def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)
    # model_svm = SVC(kernel="linear", C=1, probability=True)

    (scores_svm, mean_loss_svm, test_accuracy) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test,
                                            metric=metric)
    
    features = list(chromossome).count(1)

    if metric == 'train':
        return mean_loss_svm + 0.001*features
    
    elif metric == 'validation':

        print(f'Accuracy in Training: {mean_loss_svm}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_svm


In [8]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.95,
            iterations_size=20)
 
best_student, mean_fitness, mean_features  = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:40<00:00,  2.02s/it]


Fitness: 0.313 | features: 30
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:42<00:00,  2.15s/it]


Fitness: 0.313 | features: 30
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]


Fitness: 0.313 | features: 30
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it]


Fitness: 0.313 | features: 30
Accuracy in Training: 0.8654
Accuracy in Validation: 0.8701


0.8654

## Mulher 1

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):
    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)


    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy

def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)
    # model_svm = SVC(kernel="linear", C=1, probability=True)

    (scores_svm, mean_loss_svm, test_accuracy) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test,
                                            metric=metric)
    
    features = list(chromossome).count(1)

    if metric == 'train':
        return mean_loss_svm + 0.001*features
    
    elif metric == 'validation':

        print(f'Accuracy in Training: {mean_loss_svm}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_svm


In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.90,
            iterations_size=20)
 
best_student, mean_fitness, mean_features  = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [01:12<00:00,  3.63s/it]


Fitness: 1.04 | features: 56
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [01:03<00:00,  3.18s/it]


Fitness: 0.847 | features: 84
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Fitness: 0.847 | features: 84
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:47<00:00,  2.35s/it]


Fitness: 0.847 | features: 84
Accuracy in Training: 0.7112
Accuracy in Validation: 0.7108


0.7112

## Mulher 2

In [4]:
def run_model(model, X_train, y_train, X_test, y_test, metric):
    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)


    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy

def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)
    # model_svm = SVC(kernel="linear", C=1, probability=True)

    (scores_svm, mean_loss_svm, test_accuracy) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test,
                                            metric=metric)
    
    features = list(chromossome).count(1)

    if metric == 'train':
        return mean_loss_svm + 0.001*features
    
    elif metric == 'validation':

        print(f'Accuracy in Training: {mean_loss_svm}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_svm


In [5]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.90,
            iterations_size=20)
 
best_student, mean_fitness, mean_features  = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [01:39<00:00,  4.99s/it]


Fitness: 1.12 | features: 61
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [01:43<00:00,  5.18s/it]


Fitness: 1.12 | features: 61
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [01:24<00:00,  4.21s/it]


Fitness: 1.119 | features: 61
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [01:26<00:00,  4.31s/it]


Fitness: 1.119 | features: 61
Accuracy in Training: 0.5565
Accuracy in Validation: 0.5559


0.5565

## Mulher 3

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):
    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)


    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy

def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher3.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    model_svm = SVC(probability=True)
    # model_svm = SVC(kernel="linear", C=1, probability=True)

    (scores_svm, mean_loss_svm, test_accuracy) = run_model(model=model_svm, 
                                            X_train=X_train, 
                                            y_train=y_train, 
                                            X_test=X_test, 
                                            y_test=y_test,
                                            metric=metric)
    
    features = list(chromossome).count(1)

    if metric == 'train':
        return mean_loss_svm + 0.001*features
    
    elif metric == 'validation':

        print(f'Accuracy in Training: {mean_loss_svm}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_svm


In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student, mean_fitness, mean_features  = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [01:03<00:00,  3.15s/it]


Fitness: 0.946 | features: 19
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]


Fitness: 0.781 | features: 27
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


Fitness: 0.761 | features: 24
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:32<00:00,  1.65s/it]


Fitness: 0.761 | features: 24
Accuracy in Training: 0.7147
Accuracy in Validation: 0.7445


0.7147

# KNN

## Homem 1

In [11]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [12]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:07<00:00,  2.54it/s]


Fitness: 1.747 | features: 20
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


Fitness: 1.578 | features: 22
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:15<00:00,  1.25it/s]


Fitness: 1.468 | features: 23
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:15<00:00,  1.26it/s]


Fitness: 1.468 | features: 23
Accuracy in Training: 0.8114
Accuracy in Validation: 0.8369


0.8114

## Homem 2

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:06<00:00,  2.88it/s]


Fitness: 0.844 | features: 13
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


Fitness: 0.746 | features: 24
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.24it/s]


Fitness: 0.547 | features: 34
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


Fitness: 0.547 | features: 34
Accuracy in Training: 0.9209
Accuracy in Validation: 0.9215


0.9209

## Mulher 1

In [4]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [5]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.90,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


Fitness: 2.14 | features: 54
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


Fitness: 2.036 | features: 80
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.24it/s]


Fitness: 2.036 | features: 80
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.23it/s]


Fitness: 2.036 | features: 80
Accuracy in Training: 0.6932
Accuracy in Validation: 0.6901


0.6932

## Mulher 2

In [4]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv(f'/home/rafael/Mestrado/Artigo TLBO/database/mulher2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [5]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.90,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [02:23<?, ?it/s]


## Mulher 3

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv(f'/home/rafael/Mestrado/Artigo TLBO/database/mulher3.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()

    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Usando o KNN com 5 vizinhos (padrão)
    model_knn = KNeighborsClassifier(n_neighbors=10)

    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_knn, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.95,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness: 100%|██████████| 20/20 [00:18<00:00,  1.10it/s]


Fitness: 1.595 | features: 38
Iteration: 2


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.22it/s]


Fitness: 1.566 | features: 48
Iteration: 3


...

Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


Fitness: 1.466 | features: 51
Iteration: 20


Calculating Population Fitness: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


Fitness: 1.466 | features: 51
Accuracy in Training: 0.8076
Accuracy in Validation: 0.823


0.8076

# MLP

## Homem 1

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 21:28:13.502190: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:28:13.860845: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:28:13.860845: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:28:13.860891: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:28:13.860919: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:28:13.860919: E external/local_xla/xla/stream_

Fitness: 0.826 | features: 13
Iteration: 2


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 21:30:44.912220: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:30:45.263364: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:30:45.263366: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:30:45.263385: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:30:45.263546: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 21:30:45.263687: E external/local_xla/xla/stream_

Fitness: 0.795 | features: 19
Iteration: 3


...

Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 22:15:40.076470: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:15:40.151824: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:15:40.151889: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:15:40.156415: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:15:40.476070: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:15:40.478757: E external/local_xla/xla/stream_

Fitness: 0.726 | features: 22


2024-11-07 22:18:08.009718: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Accuracy in Training: 0.7782
Accuracy in Validation: 0.801


0.7782

In [5]:
best_student, mean_fitness, mean_features 
results = pd.DataFrame({'Iter': [i+1 for i in range(20)],
           'Mean Fitness': mean_fitness,
           'Mean Features': mean_features,
           })
results

,Iter,Mean Fitness,Mean Features
0,1,0.715425,143.55
1,2,0.696925,144.85
2,3,0.678555,144.25
3,4,0.664755,142.45
4,5,0.655210,141.95
5,6,0.650805,141.95
6,7,0.641210,139.15
7,8,0.633225,138.60
8,9,0.624070,136.95
9,10,0.620640,136.40


In [4]:
best_student, mean_fitness, mean_features 
results = pd.DataFrame({'Iter': [i+1 for i in range(20)],
           'Mean Fitness': mean_fitness,
           'Mean Features': mean_features,
           })
results

,Iter,Mean Fitness,Mean Features
0,1,0.735330,111.0
1,2,0.595630,111.0
2,3,0.596950,111.0
3,4,0.595045,111.0
4,5,0.592940,111.0
5,6,0.592715,111.0
6,7,0.597505,111.0
7,8,0.596635,111.0
8,9,0.597500,111.0
9,10,0.594380,111.0


## Homem 2

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/homem2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 22:20:58.563546: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:20:58.563577: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:20:58.924094: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:20:58.924113: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:20:58.924571: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:20:58.924883: E external/local_xla/xla/stream_

Fitness: 0.502 | features: 17
Iteration: 2


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 22:23:28.326892: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:23:28.345134: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:23:28.350051: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:23:28.357507: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:23:28.682322: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 22:23:28.684056: E external/local_xla/xla/stream_

Fitness: 0.381 | features: 26
Iteration: 3


...

Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 23:01:57.465398: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:01:57.551534: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:01:57.554018: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:01:57.554177: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:01:57.554226: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:01:57.898937: E external/local_xla/xla/stream_

Fitness: 0.293 | features: 32
Iteration: 20


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-07 23:04:10.084891: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:04:10.094199: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:04:10.094486: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:04:10.147399: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:04:10.155741: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-07 23:04:10.157089: E external/local_xla/xla/stream_

Fitness: 0.293 | features: 32


AttributeError: 'tuple' object has no attribute 'position'

In [4]:
evaluate_function(best_student[0].position, metric = 'validation')

2024-11-08 07:02:37.987153: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Accuracy in Training: 0.9147
Accuracy in Validation: 0.9292


0.9147

## Mulher 1

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher1.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.004*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.98,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 21:00:18.630483: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:00:18.683847: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:00:18.685222: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:00:18.689996: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:00:18.690432: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:00:19.021180: E external/local_xla/xla/stream_

Fitness: 0.745 | features: 16
Iteration: 2


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 21:03:35.145113: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:03:35.507457: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:03:35.507809: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:03:35.508786: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:03:35.510956: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:03:35.514913: E external/local_xla/xla/stream_

Fitness: 0.745 | features: 16
Iteration: 3


...

Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 21:49:36.660299: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:49:36.661517: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:49:37.008104: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:49:37.013289: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:49:37.013350: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 21:49:37.013862: E external/local_xla/xla/stream_

Fitness: 0.722 | features: 16


2024-11-08 21:51:59.652996: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Accuracy in Training: 0.7678
Accuracy in Validation: 0.6979


0.7678

In [ ]:
best_student.position

## Mulher 2

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher2.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.75,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-09 10:21:13.453425: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:21:13.453808: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:21:13.454180: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:21:13.454234: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:21:13.454488: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:21:13.509276: E external/local_xla/xla/stream_

Fitness: 0.916 | features: 153
Iteration: 2


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-09 10:23:44.343062: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:23:44.397465: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:23:44.406448: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:23:44.407050: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:23:44.409420: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 10:23:44.750292: E external/local_xla/xla/stream_

Fitness: 0.916 | features: 153
Iteration: 3


...

Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-09 11:08:39.756444: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 11:08:39.762406: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 11:08:39.762510: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 11:08:39.762785: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 11:08:39.767077: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-09 11:08:39.769933: E external/local_xla/xla/stream_

Fitness: 0.893 | features: 153


2024-11-09 11:11:07.136805: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Accuracy in Training: 0.7293
Accuracy in Validation: 0.7393


0.7293

In [ ]:
best_student.position

## Mulher 3

In [2]:
def run_model(model, X_train, y_train, X_test, y_test, metric):

    if metric == 'train':
        loss_scorer = make_scorer(log_loss, response_method="predict_proba")
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring=loss_scorer)
    
    elif metric == 'validation':
        scores = cross_val_score(model, X_train, y_train, cv=3)

    mean_loss = scores.mean()
    model.fit(X_train, y_train)
    mean_loss = round(mean_loss, 4)

    y_pred = model.predict(X_test)
    y_test = to_categorical(y_test, num_classes=6)

    test_accuracy = accuracy_score(y_test, y_pred)

    return scores, mean_loss, test_accuracy


def evaluate_function(chromossome, metric = 'train'):
    data = pd.read_csv('/home/rafael/Mestrado/Artigo TLBO/database/mulher3.csv', delimiter=';')
    label_counts = data['Output'].value_counts().sort_index()
    data['Output'] = data['Output'] - 1
    data = data.dropna(axis=1)
    colunas_com_infinitos = data.columns[data.isin([np.inf, -np.inf]).any()]
    data = data.drop(columns=colunas_com_infinitos)

    y = data['Output']
    X = data.drop(columns=['Output'], inplace=False)

    X = X.loc[:, np.array(chromossome, dtype=bool)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


    def create_model():
        model = Sequential([
            Dense(10, activation='tanh', input_shape=(X_train.shape[1],)),
            Dense(len(np.unique(y)), activation='softmax')
        ])
        optimizer = RMSprop(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    model_mlp = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

    y_train = to_categorical(y_train, num_classes=6)
    (scores_knn, mean_loss_knn, test_accuracy) = run_model(model=model_mlp, 
                                                           X_train=X_train, 
                                                           y_train=y_train, 
                                                           X_test=X_test, 
                                                           y_test=y_test,
                                                           metric=metric)
    
    features = list(chromossome).count(1)
    if metric == 'train':
        return mean_loss_knn + 0.001*features
        
    elif metric == 'validation':
        print(f'Accuracy in Training: {mean_loss_knn}')
        print(f'Accuracy in Validation: {round(test_accuracy, 4)}')
        return mean_loss_knn

In [3]:
tlbo = TLBO(cost_function=evaluate_function,
            population_size=20,
            dim=634,
            zero_percentage=0.95,
            iterations_size=20)
 
best_student, mean_fitness, mean_features = tlbo.execute()
evaluate_function(best_student.position, metric = 'validation')

Iteration: 1


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 07:04:38.183143: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:04:38.187495: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:04:38.525896: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:04:38.525895: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:04:38.525895: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:04:38.526261: E external/local_xla/xla/stream_

Fitness: 0.697 | features: 40
Iteration: 2


Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 07:07:04.868673: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:07:04.874765: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:07:05.212303: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:07:05.212304: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:07:05.212319: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:07:05.212342: E external/local_xla/xla/stream_

Fitness: 0.697 | features: 40
Iteration: 3


...

Calculating Population Fitness:   0%|          | 0/20 [00:00<?, ?it/s]2024-11-08 07:51:13.196400: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:51:13.200822: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:51:13.203419: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:51:13.203489: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:51:13.203489: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-08 07:51:13.223559: E external/local_xla/xla/stream_

Fitness: 0.688 | features: 40


2024-11-08 07:53:46.329064: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Accuracy in Training: 0.7457
Accuracy in Validation: 0.779


0.7457

In [ ]:
best_student.position